In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta
from datetime import datetime

In [2]:
def decode_uid(total_bh):
    mask = total_bh.uid.notnull()
    masked_total_bh = total_bh[mask]
    masked_total_bh.loc[:, "uid"] = masked_total_bh.loc[:, "uid"].str.replace("%2B", "+")
    masked_total_bh.loc[:, "uid"] = masked_total_bh.loc[:, "uid"].str.replace("%2F", "/")
    masked_total_bh.loc[:, "uid"] = masked_total_bh.loc[:, "uid"].str.replace("%3D", "=")
    masked_total_bh.loc[:, "date"] = pd.to_datetime(masked_total_bh.ht, unit = "ms")
    
    return masked_total_bh

In [5]:
order_file_loc = Path.cwd()/Path("bda2021_91APP_dataset\\bda2021_91APP_dataset\\91APP_OrderData.csv")
order_file_loc.exists()

True

In [6]:

major_order = pd.read_csv(order_file_loc)

c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
major_order.head()

,MemberID,TradesGroupCode,TradesDateTime,ChannelType,ChannelDetail,PaymentType,ShippingType,TsCount,Qty,TotalSalesAmount,TotalPrice,TotalDiscount,TotalPromotionDiscount,TotalCouponDiscount,TotalLoyaltyDiscount,Status
0,jy5E0ck6eb55UMjTCL5PCitQMn+Nf/6DfmDa+zEPtz0=,TG200831S00444,2020-08-31 16:20:37.610,OfficialECom,iOSApp,ApplePay,Home,5,5,3579.0,4540,-961.0,-861,-100.0,0,Fail
1,V7au9CyyUt7G/v00BSUQnFrxkX6aiPUrcrvUhtscl4Q=,TG201108L01031,2020-11-08 10:25:47.760,OfficialECom,iOSApp,ApplePay,Home,2,2,1280.0,1380,-100.0,0,-100.0,0,Finish
2,uxxqhgclE3hlIbm1JXmxgN0fRR57p7kqfPAX4a1Yx/0=,TG201111J01896,2020-11-11 08:25:06.460,OfficialECom,iOSApp,ApplePay,FamilyPickup,2,2,1401.0,1580,-179.0,-79,-100.0,0,Finish
3,fVspFr6aMvCmNsRevQF5X83aGRRt6S6a8lWPt2q3W40=,TG201212H00386,2020-12-12 07:26:27.823,OfficialECom,iOSApp,ApplePay,FamilyPickup,2,2,1380.0,1480,-100.0,0,-100.0,0,Finish
4,llSeF2EfgDaxvHtiTO2EPgF0tDmo47650a+uvwKpVOk=,TG201213A00357,2020-12-13 00:04:34.483,OfficialECom,iOSApp,ApplePay,Home,1,1,1912.0,1912,0.0,0,0.0,0,Finish


In [8]:
offline_mask = major_order["ChannelType"].isin(["Pos", "LocationWizard"])
finish_mask = major_order["Status"]=="Finish"
offline_order = major_order[offline_mask&finish_mask]
offline_order.TradesDateTime = pd.to_datetime(offline_order.TradesDateTime)
offline_order.head()

c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,MemberID,TradesGroupCode,TradesDateTime,ChannelType,ChannelDetail,PaymentType,ShippingType,TsCount,Qty,TotalSalesAmount,TotalPrice,TotalDiscount,TotalPromotionDiscount,TotalCouponDiscount,TotalLoyaltyDiscount,Status
7828,9jwkJdvvkJELIjpaL1tUtni1+X/60x4mywEFG1a69as=,0620160501006,2016-05-01 15:00:00,Pos,006,Cash,Store,2,2,2723.0,2960,-237.0,-237,0.0,0,Finish
7829,l90zziiHdhgfdQuqjOayDB9IJpxmpimqAjffXBgBvJc=,0420160508012,2016-05-08 18:21:00,Pos,004,Cash,Store,1,1,1380.0,1380,0.0,0,0.0,0,Finish
7830,r29S1dxlRwLDSIAF4Jw3ccK9f6atauTNh2Dq0WG527A=,0120160513108,2016-05-13 10:26:00,Pos,001,Cash,Store,1,1,0.0,0,0.0,0,0.0,0,Finish
7831,ptr8EkjpzDgZQ2y3P5dXNto6cVPtyaILyYqHzx9iT2Q=,0420160525001,2016-05-25 13:32:00,Pos,004,Cash,Store,1,1,1380.0,1380,0.0,0,0.0,0,Finish
7832,jZtRqVssj22/UTMIg3JfporhSgyjpn7zkPcPs2Vin7U=,0320160616023,2016-06-16 20:44:00,Pos,003,Cash,Store,2,2,1490.0,1500,-10.0,-10,0.0,0,Finish


In [7]:
offline_order

,MemberID,TradesGroupCode,TradesDateTime,ChannelType,ChannelDetail,PaymentType,ShippingType,TsCount,Qty,TotalSalesAmount,TotalPrice,TotalDiscount,TotalPromotionDiscount,TotalCouponDiscount,TotalLoyaltyDiscount,Status
7828,9jwkJdvvkJELIjpaL1tUtni1+X/60x4mywEFG1a69as=,0620160501006,2016-05-01 15:00:00,Pos,006,Cash,Store,2,2,2723.0,2960,-237.0,-237,0.0,0,Finish
7829,l90zziiHdhgfdQuqjOayDB9IJpxmpimqAjffXBgBvJc=,0420160508012,2016-05-08 18:21:00,Pos,004,Cash,Store,1,1,1380.0,1380,0.0,0,0.0,0,Finish
7830,r29S1dxlRwLDSIAF4Jw3ccK9f6atauTNh2Dq0WG527A=,0120160513108,2016-05-13 10:26:00,Pos,001,Cash,Store,1,1,0.0,0,0.0,0,0.0,0,Finish
7831,ptr8EkjpzDgZQ2y3P5dXNto6cVPtyaILyYqHzx9iT2Q=,0420160525001,2016-05-25 13:32:00,Pos,004,Cash,Store,1,1,1380.0,1380,0.0,0,0.0,0,Finish
7832,jZtRqVssj22/UTMIg3JfporhSgyjpn7zkPcPs2Vin7U=,0320160616023,2016-06-16 20:44:00,Pos,003,Cash,Store,2,2,1490.0,1500,-10.0,-10,0.0,0,Finish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261042,zmCy0I+B5xW/FYSPbQawST40TPH00qtc2F3l3yeE0p4=,0520210427004,2021-04-27 15:46:00,Pos,005,Cash,Store,1,1,1680.0,1680,0.0,0,0.0,0,Finish
261044,PEvidK1TyFrwLHZNsIFOgARdgpHcXcbG4yNGXt478i4=,2720210427009,2021-04-27 16:23:00,Pos,027,Cash,Store,4,4,3210.0,3460,-250.0,0,-250.0,0,Finish
261046,RTapmImK8yGwkKY/B/pCgTS1RDiVCjNgpf792Zy7UPM=,Y20210428015,2021-04-28 11:32:00,Pos,018,Cash,Store,1,1,1580.0,1580,0.0,0,0.0,0,Finish
261047,GhX4FxKa9iqQU5UJNMGXX62paxMnwmwCcyfiDH3uOwE=,3020210430016,2021-04-30 18:07:00,Pos,030,Cash,Store,1,1,1872.0,1872,0.0,0,0.0,0,Finish


In [9]:
#增加label欄
offline_order["label"] =""
offline_order.head()

c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,MemberID,TradesGroupCode,TradesDateTime,ChannelType,ChannelDetail,PaymentType,ShippingType,TsCount,Qty,TotalSalesAmount,TotalPrice,TotalDiscount,TotalPromotionDiscount,TotalCouponDiscount,TotalLoyaltyDiscount,Status,label
7828,9jwkJdvvkJELIjpaL1tUtni1+X/60x4mywEFG1a69as=,0620160501006,2016-05-01 15:00:00,Pos,006,Cash,Store,2,2,2723.0,2960,-237.0,-237,0.0,0,Finish,
7829,l90zziiHdhgfdQuqjOayDB9IJpxmpimqAjffXBgBvJc=,0420160508012,2016-05-08 18:21:00,Pos,004,Cash,Store,1,1,1380.0,1380,0.0,0,0.0,0,Finish,
7830,r29S1dxlRwLDSIAF4Jw3ccK9f6atauTNh2Dq0WG527A=,0120160513108,2016-05-13 10:26:00,Pos,001,Cash,Store,1,1,0.0,0,0.0,0,0.0,0,Finish,
7831,ptr8EkjpzDgZQ2y3P5dXNto6cVPtyaILyYqHzx9iT2Q=,0420160525001,2016-05-25 13:32:00,Pos,004,Cash,Store,1,1,1380.0,1380,0.0,0,0.0,0,Finish,
7832,jZtRqVssj22/UTMIg3JfporhSgyjpn7zkPcPs2Vin7U=,0320160616023,2016-06-16 20:44:00,Pos,003,Cash,Store,2,2,1490.0,1500,-10.0,-10,0.0,0,Finish,


# 以上為過濾order資料集

# 以下開始處理行為

In [8]:
behavior_file_loc = Path.cwd()/Path("bda2021_91APP_dataset\\bda2021_91APP_dataset\\91APP_BehaviorData")
behavior_file_loc.exists()

True

In [9]:
seven_days_bh = ['activityPageView','search','productPageView','mainPageView','categoryPageView']
ninty_days_behavior_set =set()

for bahavior_file in behavior_file_loc.iterdir():
    ninty_days_behavior_set.add(bahavior_file.name[0:-15])
    

In [10]:
ninty_days_behavior_set


{'activityPageView',
 'addToCart',
 'categoryPageView',
 'checkout',
 'mainPageView',
 'productPageView',
 'purchase',
 'search',
 'trafficSource',
 'userRegisteration'}

In [91]:
#做一個檔案table 方便之後for迴圈查找
date_table =pd.DataFrame()

date_table["file"] =""
date_table["date"] =""
for bahavior_file in behavior_file_loc.iterdir():
    cur_file_list = []
    cur_dict =pd.DataFrame()
    #print(pd.to_datetime(bahavior_file.name[-14:-4]))
    
    for n in ninty_days_behavior_set:
        cur_file_name = n+"_"+bahavior_file.name[-14:-4]+".csv"
        #print(cur_file_name)
        check_file_path = Path.cwd()/Path("bda2021_91APP_dataset\\bda2021_91APP_dataset\\91APP_BehaviorData\\"+cur_file_name)
        if check_file_path.exists():
            #print("s")
            cur_file_list.append(cur_file_name)
    cur_dict["file"] = cur_file_list
    cur_dict["date"] = pd.to_datetime(bahavior_file.name[-14:-4])
    #print(date_table)
    date_table = pd.concat([date_table, cur_dict])
    

In [92]:
date_table

,file,date
0,categoryPageView_2018-06-26.csv,2018-06-26
1,search_2018-06-26.csv,2018-06-26
2,checkout_2018-06-26.csv,2018-06-26
3,activityPageView_2018-06-26.csv,2018-06-26
4,trafficSource_2018-06-26.csv,2018-06-26
...,...,...
5,productPageView_2021-04-30.csv,2021-04-30
6,userRegisteration_2021-04-30.csv,2021-04-30
7,purchase_2021-04-30.csv,2021-04-30
8,addToCart_2021-04-30.csv,2021-04-30


In [10]:
#過濾太早沒有記錄行為的offline_order 只取紀錄行為最早日期的90天後
#bh_date_mask = offline_order["TradesDateTime"]>=date_table.iloc[1][1]+timedelta(days = 90)

#只取2021年後的order
bh_date_mask = offline_order["TradesDateTime"]>=datetime(2021,1,1)
bh_date_mask2 = offline_order["TradesDateTime"]<=datetime(2021,4,30)

new_offline_order = offline_order[bh_date_mask&bh_date_mask2]
new_offline_order

,MemberID,TradesGroupCode,TradesDateTime,ChannelType,ChannelDetail,PaymentType,ShippingType,TsCount,Qty,TotalSalesAmount,TotalPrice,TotalDiscount,TotalPromotionDiscount,TotalCouponDiscount,TotalLoyaltyDiscount,Status,label
10070,UbVqGRRv4uq06bgrqdDPLeUUgbLMLFz63W5St0JrWG0=,2420210101012,2021-01-01 17:12:00,Pos,024,Cash,Store,1,1,1980.0,1980,0.0,0,0.0,0,Finish,
10071,pIbN6MVkfXSB5mpKKz7HFjPniyVYZ+1LJFcIh/PDFH4=,1820210101004,2021-01-01 15:49:00,Pos,018,Cash,Store,6,6,4590.0,4840,-250.0,0,-250.0,0,Finish,
10072,K5fc88pkgyv336sK5wrHhhIENAX3roDC0KHLYeLUk98=,1820210102012,2021-01-02 16:10:00,Pos,018,Cash,Store,4,4,3580.0,3680,-100.0,0,-100.0,0,Finish,
10073,qOU388cad+s8+mCdcrrBjpDWVPCb0/9PXnC00ta6vBE=,0320210103042,2021-01-03 20:28:00,Pos,003,Cash,Store,3,3,3160.0,3160,0.0,0,0.0,0,Finish,
10075,Vc9UrQTwECRhsgbRFFsVVFjDKRpbFHeqE84hIZx9ru8=,0320210104024,2021-01-04 19:09:00,Pos,003,Cash,Store,2,2,1680.0,1780,-100.0,0,-100.0,0,Finish,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261039,sZIR6XRQKBt8IPIKYUqK+TvEKylc9InLnuRVilb6xb0=,2420210426013,2021-04-26 20:02:00,Pos,024,Cash,Store,1,1,1480.0,1480,0.0,0,0.0,0,Finish,
261041,H/NsV2FApl6ZCSn8vUbogdOvpbRvZnRWbykm+uv3K1c=,0520210426012,2021-04-26 18:49:00,Pos,005,Cash,Store,7,7,4440.0,4840,-400.0,0,-400.0,0,Finish,
261042,zmCy0I+B5xW/FYSPbQawST40TPH00qtc2F3l3yeE0p4=,0520210427004,2021-04-27 15:46:00,Pos,005,Cash,Store,1,1,1680.0,1680,0.0,0,0.0,0,Finish,
261044,PEvidK1TyFrwLHZNsIFOgARdgpHcXcbG4yNGXt478i4=,2720210427009,2021-04-27 16:23:00,Pos,027,Cash,Store,4,4,3210.0,3460,-250.0,0,-250.0,0,Finish,


In [14]:
df = pd.read_pickle("concate_bh_2020-10-01_sorted.pkl")
df

,dc,did,uid,ht,bh,DateTime
16002,web,d7fb867e-ea05-4da7-b9de-1f375312aeb2,IRlqIORvxbyh8BGx3xi0QnRKS31e5tOkK%2BGgaXCiQn0%3D,1601506323470,viewproduct,2020-09-30 22:52:03.470
15351,app,d6ab9029-d0c0-4064-9ab8-f425462d63aa,OWmFvhswFyiUN7PhJlaCoDJUmTqbMewavHQhZw4HKOM%3D,1601506324631,viewcategory,2020-09-30 22:52:04.631
28306,web,aab43d69-4254-4ec3-e3f6-7a5e79959ddd,%2BcvDP0cFzEim9SyjIW03xDPHdvcmwQtt6%2FJt61W%2B...,1601506325928,viewproduct,2020-09-30 22:52:05.928
11095,app,c6298c58-e77c-4ee7-a078-b83ca5fb5d42,AuUh4BPk0cWtntGuEE7ZqWZ8gLJ5%2BnYAvTHSIiGr2MI%3D,1601506327622,viewmainpage,2020-09-30 22:52:07.622
12609,web,d7fb867e-ea05-4da7-b9de-1f375312aeb2,IRlqIORvxbyh8BGx3xi0QnRKS31e5tOkK%2BGgaXCiQn0%3D,1601506327910,viewcategory,2020-09-30 22:52:07.910
...,...,...,...,...,...,...
8570,web,0dc6c50a-3248-426f-a345-01cf52a32b89,AUzOiLY%2BzKoOaM3sf%2B6Rtn5xq4ywQJ1Kvn7rXuJZT%...,1619823885448,viewmainpage,2021-04-30 23:04:45.448
3054,web,0b2f332b-d438-4754-bbdc-f6961829cad8,NaN,1619823886283,viewcategory,2021-04-30 23:04:46.283
19356,app,5115c2fb-edd6-4b3e-997f-72beeaa7e360,598aGVqXno8OIYjLatcnnqomkXre4l2Qlr85Q6uGVWY%3D,1619823892830,viewproduct,2021-04-30 23:04:52.830
18555,app,afb5c0f6-518f-4347-a121-4d6e1b1dfbc9,ydY9gwls3jxDE4NdMeLCousDxQFncnijCHWZ4dcRpuQ%3D,1619823893979,viewcategory,2021-04-30 23:04:53.979


In [15]:
temp_all_bh = decode_uid(df)

c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [16]:
temp_all_bh = temp_all_bh.drop("date", axis = 1)

In [17]:
temp_all_bh

,dc,did,uid,ht,bh,DateTime
16002,web,d7fb867e-ea05-4da7-b9de-1f375312aeb2,IRlqIORvxbyh8BGx3xi0QnRKS31e5tOkK+GgaXCiQn0=,1601506323470,viewproduct,2020-09-30 22:52:03.470
15351,app,d6ab9029-d0c0-4064-9ab8-f425462d63aa,OWmFvhswFyiUN7PhJlaCoDJUmTqbMewavHQhZw4HKOM=,1601506324631,viewcategory,2020-09-30 22:52:04.631
28306,web,aab43d69-4254-4ec3-e3f6-7a5e79959ddd,+cvDP0cFzEim9SyjIW03xDPHdvcmwQtt6/Jt61W+bGs=,1601506325928,viewproduct,2020-09-30 22:52:05.928
11095,app,c6298c58-e77c-4ee7-a078-b83ca5fb5d42,AuUh4BPk0cWtntGuEE7ZqWZ8gLJ5+nYAvTHSIiGr2MI=,1601506327622,viewmainpage,2020-09-30 22:52:07.622
12609,web,d7fb867e-ea05-4da7-b9de-1f375312aeb2,IRlqIORvxbyh8BGx3xi0QnRKS31e5tOkK+GgaXCiQn0=,1601506327910,viewcategory,2020-09-30 22:52:07.910
...,...,...,...,...,...,...
17157,app,5115c2fb-edd6-4b3e-997f-72beeaa7e360,598aGVqXno8OIYjLatcnnqomkXre4l2Qlr85Q6uGVWY=,1619823878312,viewcategory,2021-04-30 23:04:38.312
8570,web,0dc6c50a-3248-426f-a345-01cf52a32b89,AUzOiLY+zKoOaM3sf+6Rtn5xq4ywQJ1Kvn7rXuJZT+E=,1619823885448,viewmainpage,2021-04-30 23:04:45.448
19356,app,5115c2fb-edd6-4b3e-997f-72beeaa7e360,598aGVqXno8OIYjLatcnnqomkXre4l2Qlr85Q6uGVWY=,1619823892830,viewproduct,2021-04-30 23:04:52.830
18555,app,afb5c0f6-518f-4347-a121-4d6e1b1dfbc9,ydY9gwls3jxDE4NdMeLCousDxQFncnijCHWZ4dcRpuQ=,1619823893979,viewcategory,2021-04-30 23:04:53.979


In [19]:
#活動種類
activity_7 =["viewactivity", "viewcategory", "viewmainpage", "viewproduct", "search"]
activity_90 =["viewactivity", "add", "viewcategory", "checkout", "viewmainpage", "viewproduct", "purchase", "search", "traffic", "register"]


In [31]:
#過濾出VAP
result_pd =pd.DataFrame()


for i in range(new_offline_order.shape[0]):
    cur_order_time = new_offline_order.iloc[i].TradesDateTime
    cur_uid = new_offline_order.iloc[i].MemberID
    
    print(f"{i}/{new_offline_order.shape[0]}")

    
    offset_30 = cur_order_time-timedelta(days=90)
    offset_7 = cur_order_time-timedelta(days=7)
    
    #print(offset_90, offset_7)
    time_mask1_30  = temp_all_bh.DateTime >= offset_30
    time_mask1_7  = temp_all_bh.DateTime >= offset_7
    time_mask2  = temp_all_bh.DateTime <= cur_order_time
    
    cur_period_result_30 = temp_all_bh[time_mask1_30&time_mask2]
    cur_period_result_7 = temp_all_bh[time_mask1_7&time_mask2]
    
    print("Begin search")
    
    """    
    for file in cur_period_files_90.file:
        cur_bh_file = pd.read_csv(Path.cwd()/Path("bda2021_91APP_dataset\\bda2021_91APP_dataset\\91APP_BehaviorData\\"+file))
        cur_bh_file_decoded = decode_uid(cur_bh_file)
    """  
    uid_mask = cur_period_result_30.uid == cur_uid
    uid_mask2 = cur_period_result_7.uid == cur_uid
    cur_order_bh = cur_period_result_30[uid_mask]
    cur_order_bh_7 = cur_period_result_7[uid_mask2]
    print(cur_order_bh.shape, cur_order_bh_7.shape)
    cur_row = new_offline_order.iloc[i,:]
    
    if not len(cur_order_bh_7):
        if len(cur_order_bh) and cur_order_bh.bh.iloc[0] in activity_90:
            acts = cur_order_bh.bh
            #print(acts)
            #and cur_order_bh.loc[0,"bh"] in activity_90
            #print(cur_order_bh.bh.iloc[0])
            print("find user" ,cur_uid, "in",act, "labeled: A at ", offset_30)

            
            #print(cur_row)
            cur_row["label"] = "A"
            cur_row["bh_and_time"] =list(zip(acts.values, cur_order_bh.DateTime.values))
            #cur_row["act_time_period"] = , cur_order_bh.DateTime.values)
            
            #print(cur_row["bh_and_time"])
        
        else:
            print("find user" ,cur_uid, "labeled: P at", offset_7)
            cur_row["label"] = "P"
            cur_row["bh_and_time"] =""
    else:
        if cur_order_bh_7.bh.iloc[0] in activity_7:
            acts = cur_order_bh.bh

            print("find user" ,cur_uid, "labeled: V at", offset_7)

            cur_row["label"] = "V"
            #print(cur_row)
            cur_row["bh_and_time"] =list(zip(acts.values, cur_order_bh.DateTime.values))
            #cur_row["act_time_period"] = cur_order_bh.DateTime.values
            
            #print(cur_row["bh_and_time"])
        
        else:
            print("find user" ,cur_uid, "labeled: P at", offset_7)
            cur_row["label"] = "P"
            cur_row["bh_and_time"] =""
    cur_row["V"] = cur_order_bh_7.shape[0]
    cur_row["A"] = cur_order_bh.shape[0]
    result_pd = result_pd.append(cur_row)
    if(i%10) == 0:
        print("cur shape: ", result_pd.shape)

result_pd["label"] = result_pd["label"].fillna("P")
"""          
for file in cur_period_files_7.file:
    cur_bh_file = pd.read_csv(Path.cwd()/Path("bda2021_91APP_dataset\\bda2021_91APP_dataset\\91APP_BehaviorData\\"+file))
    cur_bh_file_decoded = decode_uid(cur_bh_file)
    uid_mask = cur_bh_file_decoded.uid == cur_uid
    cur_order_bh = cur_bh_file_decoded[uid_mask]
    #print(cur_order_bh.loc[0,"bh"])
    if len(cur_order_bh) and cur_order_bh.bh.iloc[0] in activity_7:
        print("find user", cur_uid, " labeled: V", "in "+ file)
        result_dict[cur_uid] = "V"
        break
"""  
            
            
    
    

0/21009
Begin search
(7, 6) (7, 6)
find user UbVqGRRv4uq06bgrqdDPLeUUgbLMLFz63W5St0JrWG0= in viewmainpage labeled: V at 2020-12-25 17:12:00
cur shape:  (1, 20)
1/21009


c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\yuzh

Begin search
(0, 6) (0, 6)
find user pIbN6MVkfXSB5mpKKz7HFjPniyVYZ+1LJFcIh/PDFH4= in viewmainpage labeled: P at 2020-12-25 15:49:00
2/21009


c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\yuzhi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Begin search
(0, 6) (0, 6)
find user K5fc88pkgyv336sK5wrHhhIENAX3roDC0KHLYeLUk98= in viewmainpage labeled: P at 2020-12-26 16:10:00
3/21009
Begin search
(0, 6) (0, 6)
find user qOU388cad+s8+mCdcrrBjpDWVPCb0/9PXnC00ta6vBE= in viewmainpage labeled: P at 2020-12-27 20:28:00
4/21009
Begin search
(0, 6) (0, 6)
find user Vc9UrQTwECRhsgbRFFsVVFjDKRpbFHeqE84hIZx9ru8= in viewmainpage labeled: P at 2020-12-28 19:09:00
5/21009
Begin search
(113, 6) (95, 6)
find user RHJzMiXJtX/+vRn4216zDM+lSCRWDzay5hUFl8n4ijQ= in viewmainpage labeled: V at 2020-12-28 11:35:00
6/21009
Begin search
(0, 6) (0, 6)
find user TlSBCM6LS7ayYLm4Hxy7E9T7zsk2N0uk07IfCx2Gtqk= in viewmainpage labeled: P at 2020-12-30 15:26:00
7/21009
Begin search
(52, 6) (18, 6)
find user xlENoQWxGQ2LuYdR7cblK2WXQgd4/T1oS5XwoLjv/Bc= in viewmainpage labeled: V at 2020-12-31 21:27:00
8/21009
Begin search
(391, 6) (146, 6)
find user +XKKb4aNg9ZzsJdR9BRt07wJdDC9o9CTWUbR3CuUbvk= in viewmainpage labeled: V at 2020-12-31 19:39:00
9/21009
Begin searc

...

Begin search
(26, 6) (26, 6)
find user 2Ao8O2ZTLjsvlsXFN7ongJQ2jeSRscO1WocVw3m3BTY= in viewmainpage labeled: V at 2021-04-11 14:39:00
20790/21009
Begin search
(157, 6) (0, 6)
find user O7Ny76VMSF6EzmMb63Pw80BSk8jAf17pnmk7bYEHC0M= in viewmainpage labeled: A at  2021-01-19 11:48:00
cur shape:  (20791, 20)
20791/21009
Begin search
(14, 6) (14, 6)
find user mhC30qiOnCn42t0XI0lok3P9QKgBHZqx2FsFpOB+lzI= in viewmainpage labeled: V at 2021-04-12 16:17:00
20792/21009
Begin search
(0, 6) (0, 6)
find user 9KQw+202AB9+j8FQ37Vvw/t/M6HlK7gWy8dGCyAIC8A= in viewmainpage labeled: P at 2021-04-12 14:20:00
20793/21009
Begin search
(0, 6) (0, 6)
find user 9el69nT95yufg6wCto56EXfiivF4SLBAg014cSJlb6A= in viewmainpage labeled: P at 2021-04-12 20:15:00
20794/21009
Begin search
(38, 6) (2, 6)
find user pn0Gvw8Yi/5J82phvX41FOiyCKt2xBGIw8G6I8sbzuo= in viewmainpage labeled: V at 2021-04-14 16:40:00
20795/21009
Begin search
(2, 6) (1, 6)
find user U6Pi4xAW9EvZ7YxxRmJr2+4EqKl57FFvsvf9Fjq6L00= in viewmainpage labele

Begin search
(22, 6) (22, 6)
find user uQeSpv8Wu+oUS7UCfympXmt0H9uyQXsmMu82bb6nm/Y= in viewmainpage labeled: V at 2021-01-18 11:24:00
20846/21009
Begin search
(0, 6) (0, 6)
find user bsoFPcIegQxNSCC/HLf96y9S5lwZIx0TvT6fTyFYv7g= in viewmainpage labeled: P at 2021-01-18 16:40:00
20847/21009
Begin search
(66, 6) (10, 6)
find user vC1M7Rz2LOHC/2OQ1g4QVZBDpZGJKzO1G8Vf8pSeQ4w= in viewmainpage labeled: V at 2021-01-20 20:07:00
20848/21009
Begin search
(31, 6) (6, 6)
find user rleipXN8uviTvJTxw2cjCCT8ygYGevIlThbh9H6I8Sc= in viewmainpage labeled: V at 2021-01-20 15:34:00
20849/21009
Begin search
(0, 6) (0, 6)
find user AWzMk9MeccFpcfXEktNDxzkdzGBZ6sbBvND5uSVZDX4= in viewmainpage labeled: P at 2021-01-20 15:03:00
20850/21009
Begin search
(0, 6) (0, 6)
find user j9aALEDhqqGRzNQ7yVw9E1a6CH5fNhAYH9rUKxiN/9Q= in viewmainpage labeled: P at 2021-01-21 20:13:00
cur shape:  (20851, 20)
20851/21009
Begin search
(33, 6) (33, 6)
find user ZqgSDsLntsPCcIRi1gwRdmcerKRVj10++TDkHczFqj4= in viewmainpage labeled

Begin search
(1, 6) (1, 6)
find user vKQNimwzXZ/uIz2LKRj2vKGGdN2reFiEvC88i2WpGl8= in viewmainpage labeled: V at 2021-02-13 17:52:00
20902/21009
Begin search
(12, 6) (0, 6)
find user KrOlYfkqd6gGle+aZYHuqUYOH3hO+cjLB1741yiKKm8= in viewmainpage labeled: A at  2020-11-23 17:44:00
20903/21009
Begin search
(26, 6) (9, 6)
find user iOTp0ShLtlM3l32YvX7txBczuar65jv9Q5CTRsFFnnY= in viewmainpage labeled: V at 2021-02-14 18:51:00
20904/21009
Begin search
(0, 6) (0, 6)
find user 6yZba5Y5+zpUUDnJRXiyQSz5oOJry8gIPv566cV0vng= in viewmainpage labeled: P at 2021-02-14 14:23:00
20905/21009
Begin search
(11, 6) (11, 6)
find user nTxnUU+g0c8goPaxkUZmaAnMkA9s43GU86IO094KzyU= in viewmainpage labeled: V at 2021-02-15 10:30:00
20906/21009
Begin search
(197, 6) (59, 6)
find user hMBeMJfd3sVE0ymWqSygcgW/eQXL74CsQHEL7z2g55M= in viewmainpage labeled: V at 2021-02-16 10:55:00
20907/21009
Begin search
(35, 6) (0, 6)
find user joOb4Re/sj2QDtYTRkMoNUjCbfDiINiuXj2VGGAa1pE= in viewmainpage labeled: A at  2020-11-25 18:

Begin search
(31, 6) (4, 6)
find user 6pCpGun57jOgsqnaTCqE//5MuKr/cGWTuIBwQOFkDT0= in viewmainpage labeled: V at 2021-03-23 17:09:00
20958/21009
Begin search
(4, 6) (4, 6)
find user dkqGGkhSdjq19dV4D45RifQza/AlBdaePFL2+nlGhf0= in viewmainpage labeled: V at 2021-03-24 12:01:00
20959/21009
Begin search
(499, 6) (26, 6)
find user gf2Q3YDlPhQpJ8CwUZrlw6KxL50hjQUALD8ekFlqKco= in viewmainpage labeled: V at 2021-03-25 15:24:00
20960/21009
Begin search
(8, 6) (0, 6)
find user kIQaH+KUrb6zoHcgzNymT6tA86oxZ/OVWSXeZpn5tpM= in viewmainpage labeled: A at  2021-01-01 21:33:00
cur shape:  (20961, 20)
20961/21009
Begin search
(2, 6) (0, 6)
find user MXaTaIjzEJlU/jgpDaYtzVQaMXnqtrJJxr8541voPl8= in viewmainpage labeled: A at  2021-01-01 11:00:00
20962/21009
Begin search
(9, 6) (9, 6)
find user AngSNBXzxDO5js6qOZPcAIj5l2SWAWIMHioZGUXoUMU= in viewmainpage labeled: V at 2021-03-26 15:11:00
20963/21009
Begin search
(0, 6) (0, 6)
find user oNj9b87ETw2qvsoRp3YDdHrCJkjIRoBYB8dyzKgIHd0= in viewmainpage labeled:

'          \nfor file in cur_period_files_7.file:\n    cur_bh_file = pd.read_csv(Path.cwd()/Path("bda2021_91APP_dataset\\bda2021_91APP_dataset\\91APP_BehaviorData\\"+file))\n    cur_bh_file_decoded = decode_uid(cur_bh_file)\n    uid_mask = cur_bh_file_decoded.uid == cur_uid\n    cur_order_bh = cur_bh_file_decoded[uid_mask]\n    #print(cur_order_bh.loc[0,"bh"])\n    if len(cur_order_bh) and cur_order_bh.bh.iloc[0] in activity_7:\n        print("find user", cur_uid, " labeled: V", "in "+ file)\n        result_dict[cur_uid] = "V"\n        break\n'

In [32]:
result_pd.to_csv("2021_VAP_7_90.csv")

In [33]:
result_pd

,A,ChannelDetail,ChannelType,MemberID,PaymentType,Qty,ShippingType,Status,TotalCouponDiscount,TotalDiscount,TotalLoyaltyDiscount,TotalPrice,TotalPromotionDiscount,TotalSalesAmount,TradesDateTime,TradesGroupCode,TsCount,V,bh_and_time,label
10070,7.0,024,Pos,UbVqGRRv4uq06bgrqdDPLeUUgbLMLFz63W5St0JrWG0=,Cash,1.0,Store,Finish,0.0,0.0,0.0,1980.0,0.0,1980.0,2021-01-01 17:12:00,2420210101012,1.0,7.0,"[(viewmainpage, 2021-01-01T07:09:46.021000000)...",V
10071,0.0,018,Pos,pIbN6MVkfXSB5mpKKz7HFjPniyVYZ+1LJFcIh/PDFH4=,Cash,6.0,Store,Finish,-250.0,-250.0,0.0,4840.0,0.0,4590.0,2021-01-01 15:49:00,1820210101004,6.0,0.0,,P
10072,0.0,018,Pos,K5fc88pkgyv336sK5wrHhhIENAX3roDC0KHLYeLUk98=,Cash,4.0,Store,Finish,-100.0,-100.0,0.0,3680.0,0.0,3580.0,2021-01-02 16:10:00,1820210102012,4.0,0.0,,P
10073,0.0,003,Pos,qOU388cad+s8+mCdcrrBjpDWVPCb0/9PXnC00ta6vBE=,Cash,3.0,Store,Finish,0.0,0.0,0.0,3160.0,0.0,3160.0,2021-01-03 20:28:00,0320210103042,3.0,0.0,,P
10075,0.0,003,Pos,Vc9UrQTwECRhsgbRFFsVVFjDKRpbFHeqE84hIZx9ru8=,Cash,2.0,Store,Finish,-100.0,-100.0,0.0,1780.0,0.0,1680.0,2021-01-04 19:09:00,0320210104024,2.0,0.0,,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261039,11.0,024,Pos,sZIR6XRQKBt8IPIKYUqK+TvEKylc9InLnuRVilb6xb0=,Cash,1.0,Store,Finish,0.0,0.0,0.0,1480.0,0.0,1480.0,2021-04-26 20:02:00,2420210426013,1.0,0.0,"[(viewmainpage, 2021-04-17T09:30:49.548000000)...",A
261041,0.0,005,Pos,H/NsV2FApl6ZCSn8vUbogdOvpbRvZnRWbykm+uv3K1c=,Cash,7.0,Store,Finish,-400.0,-400.0,0.0,4840.0,0.0,4440.0,2021-04-26 18:49:00,0520210426012,7.0,0.0,,P
261042,1.0,005,Pos,zmCy0I+B5xW/FYSPbQawST40TPH00qtc2F3l3yeE0p4=,Cash,1.0,Store,Finish,0.0,0.0,0.0,1680.0,0.0,1680.0,2021-04-27 15:46:00,0520210427004,1.0,1.0,"[(viewmainpage, 2021-04-26T13:31:04.468000000)]",V
261044,24.0,027,Pos,PEvidK1TyFrwLHZNsIFOgARdgpHcXcbG4yNGXt478i4=,Cash,4.0,Store,Finish,-250.0,-250.0,0.0,3460.0,0.0,3210.0,2021-04-27 16:23:00,2720210427009,4.0,24.0,"[(viewmainpage, 2021-04-27T09:30:29.626000000)...",V


In [119]:
#只取20210101 -90天內的紀錄
bh2021_mask = date_table.date>=datetime(2021,4,15)-timedelta(days =14)
date_table_2021 = date_table[bh2021_mask]


In [125]:
temp_all_bh = pd.DataFrame()
count = 0
for file in date_table_2021.file:
    cur_bh_file = pd.read_csv(Path.cwd()/Path("bda2021_91APP_dataset\\bda2021_91APP_dataset\\91APP_BehaviorData\\"+file))
    cur_bh_file_decoded = decode_uid(cur_bh_file)
    temp_all_bh = pd.concat([temp_all_bh,cur_bh_file_decoded])
    count+=1
    
    if count%200 ==0:
        temp_all_bh.to_csv("temp_all_bh.csv")
        
temp_all_bh.to_csv("temp_all_bh.csv")     

c:\users\pppoe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\users\pppoe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [72]:
new_offline_order.head()

,MemberID,TradesGroupCode,TradesDateTime,ChannelType,ChannelDetail,PaymentType,ShippingType,TsCount,Qty,TotalSalesAmount,TotalPrice,TotalDiscount,TotalPromotionDiscount,TotalCouponDiscount,TotalLoyaltyDiscount,Status,label
9311,rayOc0FOiTyJHJPqrrJ2f18VdDMD4aqliPpfWqKsKxY=,2420200101011,2020-01-01 16:50:00,Pos,024,Cash,Store,2,2,1501.0,1501,0.0,0,0.0,0,Finish,
9312,BefEMz8G5OCDMlgqVMnJFj210E7T77mosy3yEkDaNlI=,2420200102013,2020-01-02 20:38:00,Pos,024,Cash,Store,1,1,0.0,0,0.0,0,0.0,0,Finish,
9314,EXx+fILg1qkJ1LofEqw9gA3ArFeHvusHYzoJSjjzs9Y=,0520200103007,2020-01-03 18:13:00,Pos,005,Cash,Store,1,1,1380.0,1380,0.0,0,0.0,0,Finish,
9316,eLgjI1/5G/TgY+ek4ntibr7PoK2KDrmQZgnRb5c3ssA=,A1200103A-029,2020-01-03 21:54:00,Pos,021,Cash,Store,1,1,2380.0,2380,0.0,0,0.0,0,Finish,
9317,+MsIpC7qnKVNv37Lj0W2Ja6AmGY93Yaj0RyD/8XXG8g=,1820200104009,2020-01-04 18:55:00,Pos,018,Cash,Store,1,1,1880.0,1880,0.0,0,0.0,0,Finish,


In [ ]:
#過濾出VAP
for i in new_offline_order.head().iterrows():
    #print(i)
    cur_order_time = i[1]["TradesDateTime"]
    cur_uid = i[1]["MemberID"]
    #print(cur_order_time)
    
    #過濾出A 填入A
    offset_90 = cur_order_time-timedelta(days=90)
    time_mask1  = total_bh.date >= offset_90
    time_mask2  = total_bh.date <= cur_order_time
    uid_mask = total_bh.uid == cur_uid
    act_mask = total_bh.bh.isin(activity_90)
    total_bh[(time_mask1 & time_mask2 & uid_mask & act_mask)].label = "A"
    
    #過濾出V 填入V
    offset_7 = cur_order_time-timedelta(days=7)
    time_mask1_7  = total_bh.date >= offset_7
    time_mask2_7  = total_bh.date <= cur_order_time
    uid_mask = total_bh.uid == cur_uid
    act_mask_7 = total_bh.bh.isin(activity_7)
    total_bh[(time_mask1_7 & time_mask2_7 & uid_mask & act_mask_7)].label = "V"
    #print(cur_period_files)
    
total_bh.label = total_bh.label.fillna(value = "P")
total_bh.to_csv("VAP_filtered.csv")

In [40]:
fix = pd.read_csv("2021_VAP_7_90.csv", index_col=0)


In [41]:
fix.TradesDateTime

10070     2021-01-01 17:12:00.000
10071     2021-01-01 15:49:00.000
10072     2021-01-02 16:10:00.000
10073     2021-01-03 20:28:00.000
10075     2021-01-04 19:09:00.000
                   ...           
261039    2021-04-26 20:02:00.000
261041    2021-04-26 18:49:00.000
261042    2021-04-27 15:46:00.000
261044    2021-04-27 16:23:00.000
261046    2021-04-28 11:32:00.000
Name: TradesDateTime, Length: 21009, dtype: object

In [34]:
fix.TradesDateTime = new_offline_order.TradesDateTime

In [39]:
fix.to_csv("2021_VAP_7_90.csv")